# Notebook 4: Robust Model Predictive Control System

**Project:** `PharmaControl-Pro`
**Goal:** Build the 'brain' of our control system. This involves creating a robust Model Predictive Control (MPC) class that encapsulates the logic for real-time decision-making. This controller will use the trained model to find the best control action while respecting process constraints.

### Table of Contents
1. [The MPC Controller: Architecture and Logic](#1.-The-MPC-Controller:-Architecture-and-Logic)
2. [Implementing the MPC Controller Class](#2.-Implementing-the-MPC-Controller-Class)
3. [Defining Process Constraints](#3.-Defining-Process-Constraints)
4. [Standalone Test of the MPC Controller](#4.-Standalone-Test-of-the-MPC-Controller)

--- 
## 1. The MPC Controller: Architecture and Logic

The MPC controller's job is to answer one question at each decision point: **"Given the current state of the plant, what is the best possible action to take right now to meet our future targets?"**

To do this, it follows a precise, multi-step algorithm based on the principles we discussed in Notebook 1:

1.  **Generate Candidate Actions:** Create a 'lattice' or grid of potential future control plans. For example, 'Plan A: increase spray rate by 5%', 'Plan B: hold steady', 'Plan C: decrease air flow by 10 m³/h', and so on. This is our search space.

2.  **Enforce Constraints:** This is a critical safety and quality step. The controller must filter out any candidate plans that are physically impossible or would violate process limits (e.g., a negative flow rate, a change that is too rapid, or a value outside the equipment's operational range).

3.  **Predict and Evaluate:** For every *valid* candidate plan, use our trained `GranulationPredictor` model to forecast the future CMAs over the horizon `H`. Then, calculate a 'cost' for each prediction. The cost function is key:
    *   It heavily penalizes deviations from the target CMAs (`target_error_cost`).
    *   It can also lightly penalize large or rapid changes in the control actions themselves to promote smooth, stable operation (`control_effort_cost`).
    *   `Total Cost = target_error_cost + λ * control_effort_cost`

4.  **Select Optimal Action:** The candidate plan with the lowest total cost is the winner.

5.  **Apply Receding Horizon:** The controller returns only the *very first step* of the winning plan. This action is sent to the plant. At the next decision point, the entire process is repeated, ensuring the controller is always reacting to the latest information.

--- 
## 2. Implementing the MPC Controller Class

We will encapsulate all this logic into a reusable `MPCController` class. This class will be defined in `src/mpc_controller.py`.

--- 
## 3. Defining Process Constraints

A real industrial process has strict limits. The controller must never be allowed to suggest an action that could damage the equipment or ruin the product. We will define these limits in a clear, structured dictionary that our `MPCController` can use.

In [5]:
PROCESS_CONSTRAINTS = {
    'spray_rate': {
        'min_val': 80.0,              # Equipment minimum
        'max_val': 180.0,             # Equipment maximum
        'max_change_per_step': 10.0   # Max allowed change in one go (for stability)
    },
    'air_flow': {
        'min_val': 400.0,
        'max_val': 700.0,
        'max_change_per_step': 25.0
    },
    'carousel_speed': {
        'min_val': 20.0,
        'max_val': 40.0,
        'max_change_per_step': 2.0
    }
}

--- 
## 4. Standalone Test of the MPC Controller

Before we connect the controller to our simulator in the final notebook, let's perform a standalone test. We will create some dummy historical data, define a target, and ask the controller for a single best action. This allows us to debug the controller's logic in isolation.

In [6]:
import torch
import joblib
import os, sys
import pandas as pd
import numpy as np
from V1.src.model_architecture import GranulationPredictor
from V1.src.mpc_controller import MPCController

# --- Load all necessary components ---
DATA_DIR = '../data'
MODEL_SAVE_PATH = os.path.join(DATA_DIR, 'best_predictor_model.pth')
SCALER_FILE = os.path.join(DATA_DIR, 'model_scalers.joblib')  # FIXED: Correct scaler file name
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# FIXED: Load model configuration from saved checkpoint
print("Loading trained model and configuration...")
checkpoint = torch.load(MODEL_SAVE_PATH, map_location=DEVICE)

# Extract saved configuration and hyperparameters
saved_config = checkpoint['config']
saved_hparams = checkpoint['hyperparameters']

CMA_COLS = saved_config['CMA_COLS']
CPP_COLS_BASE = ['spray_rate', 'air_flow', 'carousel_speed']  # Base control variables
CPP_COLS_FULL = saved_config['CPP_COLS']  # Includes soft sensors

print(f"Model architecture: {saved_hparams['d_model']}-dim transformer")
print(f"  Encoder layers: {saved_hparams['num_encoder_layers']}")
print(f"  Decoder layers: {saved_hparams['num_decoder_layers']}")
print(f"  Attention heads: {saved_hparams['nhead']}")
print(f"  Dropout: {saved_hparams['dropout']:.3f}")

# Create model with correct hyperparameters from checkpoint
model = GranulationPredictor(
    cma_features=len(CMA_COLS),
    cpp_features=len(CPP_COLS_FULL),
    d_model=saved_hparams['d_model'],
    nhead=saved_hparams['nhead'],
    num_encoder_layers=saved_hparams['num_encoder_layers'],
    num_decoder_layers=saved_hparams['num_decoder_layers'],
    dropout=saved_hparams['dropout']
)

# Load the trained model weights
model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)
model.eval()

# Load scalers
scalers = joblib.load(SCALER_FILE)
print(f"Loaded scalers for {len(scalers)} variables")

# --- MPC Configuration ---
MPC_CONFIG = {
    'horizon': saved_config['HORIZON'],  # Use same horizon as training
    'cpp_names': CPP_COLS_BASE,
    'cma_names': CMA_COLS,
    'cpp_names_and_soft_sensors': CPP_COLS_FULL,
    'discretization_steps': 3, # Use 3 steps for each CPP delta: [-max, 0, +max]
    'control_effort_lambda': 0.05 # Small penalty for control changes
}

# --- Instantiate the Controller ---
mpc = MPCController(model, MPC_CONFIG, PROCESS_CONSTRAINTS, scalers)
print("✅ MPC Controller instantiated successfully with correct model configuration.")

Loading trained model and configuration...
Model architecture: 64-dim transformer
  Encoder layers: 4
  Decoder layers: 4
  Attention heads: 4
  Dropout: 0.172
Loaded scalers for 7 variables
✅ MPC Controller instantiated successfully with correct model configuration.


In [7]:
# --- Prepare Realistic Test Data ---

# Create realistic historical data (36 steps) with process dynamics
# Simulate a process that has been slowly drifting from target
print("Generating realistic test data with process dynamics...")

# Initialize with steady state, then add realistic variations
base_d50 = 450.0
base_lod = 1.2
base_spray = 130.0
base_air = 550.0
base_carousel = 30.0

# Generate realistic time series with trends and noise
np.random.seed(42)  # For reproducible test results
time_steps = 36

# Create slow process drift (simulating equipment aging/fouling)
drift_d50 = np.linspace(0, 15, time_steps)  # Gradual increase in particle size
drift_lod = np.linspace(0, 0.3, time_steps)  # Gradual increase in moisture

# Add realistic process noise
noise_scale_d50 = 8.0
noise_scale_lod = 0.08
noise_scale_spray = 3.0
noise_scale_air = 15.0
noise_scale_carousel = 0.8

# Generate realistic historical data with autocorrelation
d50_history = []
lod_history = []
spray_history = []
air_history = []
carousel_history = []

for i in range(time_steps):
    # Add autocorrelated noise (process has memory)
    if i == 0:
        d50_noise = np.random.normal(0, noise_scale_d50)
        lod_noise = np.random.normal(0, noise_scale_lod)
        spray_noise = np.random.normal(0, noise_scale_spray)
        air_noise = np.random.normal(0, noise_scale_air)
        carousel_noise = np.random.normal(0, noise_scale_carousel)
    else:
        # Autocorrelated noise (0.7 correlation with previous step)
        d50_noise = 0.7 * d50_noise + 0.3 * np.random.normal(0, noise_scale_d50)
        lod_noise = 0.7 * lod_noise + 0.3 * np.random.normal(0, noise_scale_lod)
        spray_noise = 0.7 * spray_noise + 0.3 * np.random.normal(0, noise_scale_spray)
        air_noise = 0.7 * air_noise + 0.3 * np.random.normal(0, noise_scale_air)
        carousel_noise = 0.7 * carousel_noise + 0.3 * np.random.normal(0, noise_scale_carousel)
    
    # Apply process constraints and realistic bounds
    d50_val = max(300, min(600, base_d50 + drift_d50[i] + d50_noise))
    lod_val = max(0.5, min(3.0, base_lod + drift_lod[i] + lod_noise))
    spray_val = max(85, min(175, base_spray + spray_noise))
    air_val = max(420, min(680, base_air + air_noise))
    carousel_val = max(22, min(38, base_carousel + carousel_noise))
    
    d50_history.append(d50_val)
    lod_history.append(lod_val)
    spray_history.append(spray_val)
    air_history.append(air_val)
    carousel_history.append(carousel_val)

# Convert to arrays
dummy_past_cmas_unscaled = np.column_stack([d50_history, lod_history])
dummy_past_cpps_unscaled = np.column_stack([spray_history, air_history, carousel_history])

# Calculate soft sensors for the complete feature set
dummy_se = (dummy_past_cpps_unscaled[:, 0] * dummy_past_cpps_unscaled[:, 2]) / 1000.0
dummy_fr = (dummy_past_cpps_unscaled[:, 2]**2) / 9.81
dummy_past_cpps_full_unscaled = np.hstack([
    dummy_past_cpps_unscaled,
    dummy_se.reshape(-1, 1),
    dummy_fr.reshape(-1, 1)
])

# Convert to DataFrames as expected by suggest_action method
past_cmas_df = pd.DataFrame(dummy_past_cmas_unscaled, columns=CMA_COLS)
past_cpps_df = pd.DataFrame(dummy_past_cpps_full_unscaled, columns=CPP_COLS_FULL)

# Define a challenging but achievable target - smaller, wetter granules
target_d50 = 350.0  # Reduce particle size by ~100 μm
target_lod = 1.8    # Increase moisture by ~0.3%
target_cmas_unscaled = np.tile([target_d50, target_lod], (MPC_CONFIG['horizon'], 1))

# Display current state and target
current_d50 = past_cmas_df.iloc[-1]['d50']
current_lod = past_cmas_df.iloc[-1]['lod']
current_spray = past_cpps_df.iloc[-1]['spray_rate']
current_air = past_cpps_df.iloc[-1]['air_flow']
current_carousel = past_cpps_df.iloc[-1]['carousel_speed']

print(f"\n📊 Process History Summary:")
print(f"  d50 range: {past_cmas_df['d50'].min():.1f} - {past_cmas_df['d50'].max():.1f} μm (trend: +{past_cmas_df['d50'].iloc[-1] - past_cmas_df['d50'].iloc[0]:.1f})")
print(f"  LOD range: {past_cmas_df['lod'].min():.3f} - {past_cmas_df['lod'].max():.3f} % (trend: +{past_cmas_df['lod'].iloc[-1] - past_cmas_df['lod'].iloc[0]:.3f})")

print(f"\n🎯 Control Challenge:")
print(f"  Current State: d50={current_d50:.1f} μm, LOD={current_lod:.3f} %")
print(f"  Target State:  d50={target_d50:.1f} μm, LOD={target_lod:.3f} %")
print(f"  Required Change: d50 {current_d50-target_d50:+.1f} μm, LOD {target_lod-current_lod:+.3f} %")

print(f"\n⚙️ Current Operating Point:")
print(f"  Spray Rate: {current_spray:.1f} g/min")
print(f"  Air Flow: {current_air:.1f} m³/h") 
print(f"  Carousel Speed: {current_carousel:.1f} rpm")

# --- Run the MPC Decision Logic ---
print(f"\n🔄 Running MPC optimization...")
suggested_action = mpc.suggest_action(past_cmas_df, past_cpps_df, target_cmas_unscaled)

print("\n🎯 MPC Recommendation:")
print("To reach the target, the optimal immediate action is:")
for i, name in enumerate(MPC_CONFIG['cpp_names']):
    current_val = past_cpps_df.iloc[-1][name]
    suggested_val = suggested_action[i]
    change = suggested_val - current_val
    print(f"  {name:15}: {current_val:6.1f} → {suggested_val:6.1f} ({change:+6.1f})")

# Physical interpretation
print("\n🧠 Physical Interpretation:")
spray_change = suggested_action[0] - current_spray
air_change = suggested_action[1] - current_air
carousel_change = suggested_action[2] - current_carousel

if spray_change < -1:
    print(f"  ↓ Reduce spray rate by {abs(spray_change):.1f} g/min → smaller particles")
elif spray_change > 1:
    print(f"  ↑ Increase spray rate by {spray_change:.1f} g/min → larger particles")
else:
    print(f"  → Maintain spray rate (minimal change: {spray_change:+.1f} g/min)")

if air_change < -5:
    print(f"  ↓ Reduce air flow by {abs(air_change):.1f} m³/h → less drying, higher moisture")
elif air_change > 5:
    print(f"  ↑ Increase air flow by {air_change:.1f} m³/h → more drying, lower moisture")
else:
    print(f"  → Maintain air flow (minimal change: {air_change:+.1f} m³/h)")

if carousel_change > 0.5:
    print(f"  ↑ Increase carousel speed by {carousel_change:.1f} rpm → less residence time, higher moisture")
elif carousel_change < -0.5:
    print(f"  ↓ Reduce carousel speed by {abs(carousel_change):.1f} rpm → more residence time, lower moisture")
else:
    print(f"  → Maintain carousel speed (minimal change: {carousel_change:+.1f} rpm)")

print(f"\n✅ MPC controller test completed successfully with realistic process data!")

Generating realistic test data with process dynamics...

📊 Process History Summary:
  d50 range: 450.1 - 466.4 μm (trend: +11.4)
  LOD range: 1.189 - 1.535 % (trend: +0.323)

🎯 Control Challenge:
  Current State: d50=465.3 μm, LOD=1.512 %
  Target State:  d50=350.0 μm, LOD=1.800 %
  Required Change: d50 +115.3 μm, LOD +0.288 %

⚙️ Current Operating Point:
  Spray Rate: 132.1 g/min
  Air Flow: 547.7 m³/h
  Carousel Speed: 30.7 rpm

🔄 Running MPC optimization...


Evaluating MPC Candidates:   0%|          | 0/27 [00:00<?, ?it/s]


🎯 MPC Recommendation:
To reach the target, the optimal immediate action is:
  spray_rate     :  132.1 →  122.1 ( -10.0)
  air_flow       :  547.7 →  547.7 (  +0.0)
  carousel_speed :   30.7 →   32.7 (  +2.0)

🧠 Physical Interpretation:
  ↓ Reduce spray rate by 10.0 g/min → smaller particles
  → Maintain air flow (minimal change: +0.0 m³/h)
  ↑ Increase carousel speed by 2.0 rpm → less residence time, higher moisture

✅ MPC controller test completed successfully with realistic process data!


### Analysis of the Suggestion

To reach a target of smaller (`d50`=350) and wetter (`LOD`=1.8) granules from a state of larger, drier ones, we expect the controller to suggest actions that:
1.  **Reduce Granule Size:** This is primarily achieved by decreasing the `spray_rate`.
2.  **Increase Moisture:** This can be done by decreasing the `air_flow` or increasing the `carousel_speed` (reducing drying time).

Check if the controller's suggested action aligns with this physical intuition. This confirms that the model has learned meaningful relationships and the MPC logic is working correctly.

We are now ready to put all the pieces together in the final notebook and run a full closed-loop simulation.